<a href="https://colab.research.google.com/github/aaronjoseph/NLP_Boiler_Plate/blob/main/Sentiment_Data_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install mlflow
#Load the libraries
from mlflow import log_metric, log_param, log_artifacts
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
from tqdm import tqdm
import collections, numpy
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
tqdm.pandas()
from sklearn.model_selection import train_test_split,KFold
nltk.download('stopwords')
! pip install neptune-client==0.4.132
import neptune
neptune.init(
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOGI1M2QwMmItZGQyMi00MTBmLThhZTktZDY0OThjYmZlMGMyIn0=",
    project_qualified_name="aaronjosephmathew/sandbox"
)

     |████████████████████████████████| 14.2MB 8.5MB/s 
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 1.1MB 60.9MB/s 
     |████████████████████████████████| 153kB 54.1MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 348kB 51.2MB/s 
     |████████████████████████████████| 163kB 63.0MB/s 
     |████████████████████████████████| 481kB 47.6MB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
     |████████████████████████████████| 133kB 52.1MB/s 
     |████████████████████████████████| 2.6MB 48.9MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=7d5104164160c9ee59ba73b04e6f690859daa05d5ffcce4eaacdfade440fa1ab
  Stored in directory: /root/.cache/pip/wheels/84/07/f7/12f7370ca47a

Project(aaronjosephmathew/sandbox)

In [3]:
sentiment_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon.csv',header=None,error_bad_lines=False,engine='python')
#sentiment_data = sentiment_data.sample(n=1_00_000,replace=True)

In [4]:
# Data Cleaning Process
def preprocess(text):
  text=text.lower()
  # remove hyperlinks
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
  #remove hashtag sign
  text=re.sub(r"#","",text)   
  #remove mentions
  text = re.sub(r"(?:\@)\w+", '', text)
  #remove non ascii chars
  text=text.encode("ascii",errors="ignore").decode()
  #remove some puncts (except . ! ?)
  text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
  text=re.sub(r'[!]+','!',text)
  text=re.sub(r'[?]+','?',text)
  text=re.sub(r'[.]+','.',text)
  text=re.sub(r"'","",text)
  text=re.sub(r"\(","",text)
  text=re.sub(r"\)","",text)    
  text=" ".join(text.split())
  return text

def simple_stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text= ' '.join([ps.stem(word) for word in text.split()])
  return text

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
  #set stopwords to english
  stop=set(stopwords.words('english'))
  # Tokenization
  tokenizer=ToktokTokenizer()
  #Setting English stopwords
  stopword_list=nltk.corpus.stopwords.words('english')
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
      filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
      filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)    
  return filtered_text

In [5]:
#Apply function
sentiment_data['clean']=sentiment_data[5].progress_apply(preprocess)
sentiment_data['stem']=sentiment_data['clean'].progress_apply(simple_stemmer)
sentiment_data['token'] = sentiment_data['stem'].progress_apply(remove_stopwords) 

100%|██████████| 1600000/1600000 [08:33<00:00, 3113.33it/s]


In [6]:
from sklearn.metrics import f1_score, precision_score, recall_score, auc, roc_curve, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()
sentiment_data['output'] = le.fit_transform(sentiment_data[0])
# Splitting the data - Into train_test
X = sentiment_data['token']
y = sentiment_data['output']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42) 

In [8]:
# Count Vectorizer for Bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(X_train)
#transformed test reviews
cv_test_reviews=cv.transform(X_test)
# TFIDF
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(X_train)
#transformed test reviews
tv_test_reviews=tv.transform(X_test)

### Reference Data

Train Data | Test Data
---| ---
Count Vectorizer Data
cv_train_reviews | y_train
cv_test_reviews | y_test
TFIDF Data | 
tv_train_reviews | y_train
tv_test_reviews | y_test

AUTOML 

- PyCaret

PyCaret is an open source, low-code machine learning library in Python that allows you to go from preparing your data to deploying your model within seconds in your choice of notebook environment.

PyCaret being a low-code library makes you more productive. With less time spent coding, you and your team can now focus on business problems.

PyCaret is simple and easy to use machine learning library that will help you to perform end-to-end ML experiments with less lines of code.

PyCaret is a business ready solution. It allows you to do prototyping quickly and efficiently from your choice of notebook environment.

     |████████████████████████████████| 256kB 13.8MB/s 
     |████████████████████████████████| 157.5MB 43kB/s 
     |████████████████████████████████| 276kB 56.0MB/s 
     |████████████████████████████████| 6.8MB 55.0MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 65.8MB 83kB/s 
     |████████████████████████████████| 245kB 58.9MB/s 
     |████████████████████████████████| 1.8MB 52.1MB/s 
     |████████████████████████████████| 1.6MB 57.9MB/s 
     |████████████████████████████████| 174kB 53.4MB/s 
     |████████████████████████████████| 2.1MB 42.9MB/s 
     |████████████████████████████████| 604kB 51.5MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 3.1MB 49.5MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 296kB 68.6MB/s 
  Created wheel for pyod: filename=pyod-0.8.6-cp36-no

ImportError: ignored